# Day 16

https://adventofcode.com/2021/day/16

Packet object chops off bits from the start of the code as it needs them, parses, then creates sub-Packets and hands off the rest of the code to them to do the same. End with a tree of Packet objects from which you can recursively calculate the answers. 

In [1]:
import numpy as np

In [2]:
hex2bin_dict = {
    "0": "0000",
    "1": "0001",
    "2": "0010",
    "3": "0011",
    "4": "0100",
    "5": "0101",
    "6": "0110",
    "7": "0111",
    "8": "1000",
    "9": "1001",
    "A": "1010",
    "B": "1011",
    "C": "1100",
    "D": "1101",
    "E": "1110",
    "F": "1111"
}

def hex2bin(h):
    return ''.join([hex2bin_dict[x] for x in h])

def bin2dec(b):
    return np.sum(np.array(list(b), dtype = "int") * np.power(2, range(len(b) - 1, -1, -1), dtype = np.int64))

In [3]:
def f_lt(l):
    if len(l) != 2:
        raise Exception("too many arguments")
    return np.int64(l[0] < l[1])

def f_gt(l):
    if len(l) != 2:
        raise Exception("too many arguments")
    return np.int64(l[0] > l[1])

def f_eq(l):
    if len(l) != 2:
        raise Exception("too many arguments")
    return np.int64(l[0] == l[1])

def f_literal(l):
    raise Exception("trying to evaluate a literal as an operator")

type2function = {
    0: np.sum,
    1: np.product,
    2: np.min,
    3: np.max,
    4: f_literal,
    5: f_gt,
    6: f_lt,
    7: f_eq
}

In [4]:
with open("input_16.txt", "r") as f:
    raw = f.read()
    
code = hex2bin(raw)

In [5]:
class Packet:
    
    def __init__(self, code):
        self.code = code
        self.add_metadata()
        
        if self.is_operator:
            self.evaluate_sub_packets()
            self.evaluate_operator()
        else:
            self.sub_packets = []
            self.evaluate_literal()
        
        
    def add_metadata(self):
        self.version = bin2dec(self.next_n_bits(3))
        self.type = bin2dec(self.next_n_bits(3))
        self.is_operator = self.type != 4
        self.f = type2function[self.type]
        
        if self.is_operator:
            self.length_type_id = bin2dec(self.next_n_bits(1))
            
            if self.length_type_id == 1:
                self.length_bits = 11
            else: 
                self.length_bits = 15
            
            self.length = bin2dec(self.next_n_bits(self.length_bits))
            
            
    def evaluate_literal(self):
        res = ''
        next_bit = 1
        while next_bit == 1:
            next_bit = bin2dec(self.next_n_bits(1))
            res = res + self.next_n_bits(4)
            
        self.literal_value = bin2dec(res)
            
    def evaluate_operator(self):
        literal_values = np.array([x.literal_value for x in self.sub_packets], dtype = np.int64)
        self.literal_value = self.f(literal_values)
            
    def is_finished(self):
        return np.all(np.array(list(self.code), dtype = "str") == "0")
            
    def evaluate_sub_packets(self):
        self.sub_packets = []
        
        if self.length_type_id == 0:
            sub_packet_bits = self.next_n_bits(self.length)
            new_packet = Packet(sub_packet_bits)
            self.sub_packets.append(new_packet)
            
            while not new_packet.is_finished():
                new_packet = Packet(new_packet.code)
                self.sub_packets.append(new_packet)
                
        else:
            for i in range(self.length):
                new_packet = Packet(self.code)
                self.code = new_packet.code
                self.sub_packets.append(new_packet)
        
            
    def next_n_bits(self, n):
        res, self.code = (self.code[:n], self.code[n:])
        return res

In [6]:
p = Packet(code)

In [7]:
def total_version(p):
    res = p.version
    res = res + np.sum([total_version(x) for x in p.sub_packets], dtype = "int")
    return res

total_version(p)

1014

In [8]:
p.literal_value

1922490999789